<a href="https://colab.research.google.com/github/matthewreader/continuous-learning/blob/main/projects/manning-liveprojects/Hate_Speech_Detection_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import libraries and load the hate speech data**

In [3]:
# 1. A cell containing imports of all the libraries.
!pip install transformers
!pip install SentencePiece
!pip install pytorch_lightning

import pandas as pd
import regex as re
from google.colab import drive
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import functional as F
from transformers import AlbertTokenizer, AlbertModel 
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.5 MB/s 
     |████████████████████████████████| 925 kB 5.2 MB/s 
     |████████████████████████████████| 829 kB 33.3 MB/s 
     |████████████████████████████████| 125 kB 38.0 MB/s 
     |████████████████████████████████| 282 kB 53.1 MB/s 
     |████████████████████████████████| 1.3 MB 37.5 MB/s 
     |████████████████████████████████| 160 kB 49.5 MB/s 
     |████████████████████████████████| 271 kB 45.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=acbd1b91b46c591

In [4]:
# 2. A cell containing code for mounting your Google Drive in Colab.
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# 3. A cell containing code for reading the data using pandas.
GDRIVE_DATA = "gdrive/My Drive/Colab Notebooks/Data/"
tweets = pd.read_csv(GDRIVE_DATA + "Khilnani_LP_hate_speech_data.csv")
tweets.head()

,Unnamed: 0,tweet,class
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,1,""" momma said no pussy cats inside my doghouse """,0
2,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://...",0
3,3,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo...",0
4,4,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these...",0


## **Remove words starting with an @ sign**

Usernames are represented in our Twitter data as strings beginning with the "@" symbol.  It does not make sense to train our model on usernames, so we first need to remove usernames from our tweets.  We can do so using the regex module as suggested by the Manning liveProject, however I think it's a little more straightforward to use `pandas.Series.str.replace` since our data is in a `pandas` dataframe.


In [6]:
# 4. Cells containing code for writing a function to clean the text data and
# then applying that function on the dataset.
tweets["tweet_clean"] = tweets["tweet"].str.replace("@\w+", "")
tweets.head()

,Unnamed: 0,tweet,class,tweet_clean
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0,!!! RT : As a woman you shouldn't complain abo...
1,1,""" momma said no pussy cats inside my doghouse """,0,""" momma said no pussy cats inside my doghouse """
2,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://...",0,""": -SimplyAddictedToGuys http://t.co/1jL4hi8ZM..."
3,3,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo...",0,""": http://t.co/3gzUpfuMev"" woof woof and hot s..."
4,4,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these...",0,""": Lemmie eat a Oreo &amp; do these dishes."" O..."


That's a bit better, but we still have a lot that probably should be cleaned up.  Knowing that this is Twitter data, any kind of hyperlink could be safely removed as well.  Punctuation should probably be removed.  Hashtags might be important if they contain any kind of hateful phrase.

In [7]:
# 5. Cells containing code for splitting the data into train and validation
# sets using sklearn.
X_train, X_test, y_train, y_test = train_test_split(
    tweets["tweet_clean"].values,
    tweets["class"].values
)

In [8]:
# 6. Cells containing code for loading the pre-trained AlbertTokenizer
# and tokenizing the data.
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

train_tokens = tokenizer(
    list(X_train),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=64
)

val_tokens = tokenizer(
    list(X_test),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=64
)

device = "cuda"

train_set = [train_tokens["input_ids"].to(device),
             train_tokens["attention_mask"].to(device),
             train_tokens["token_type_ids"].to(device),
             torch.tensor(y_train).to(device)] 

val_set = [val_tokens["input_ids"].to(device),
           val_tokens["attention_mask"].to(device),
           val_tokens["token_type_ids"].to(device),
           torch.tensor(y_test).to(device)] 

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [9]:
# 7. A cell containing the code for the DataModule class built in 
# PyTorch Lightning

BATCH_SIZE = 32

class ClassificationData(pl.LightningDataModule):
    def __init__(self, train_set, val_set):
        super().__init__()

        self.train_set = DataLoader(
            TensorDataset(*train_set),
            batch_size=BATCH_SIZE)
        self.val_set = DataLoader(
            TensorDataset(*val_set),
            batch_size=BATCH_SIZE)

    def train_dataloader(self):
      return self.train_set

    def val_dataloader(self):
      return self.val_set

dls = ClassificationData(train_set, val_set)
next(iter(dls.train_set))

[tensor([[   2, 1399,  263,  ...,    0,    0,    0],
         [   2,   32,   22,  ...,    0,    0,    0],
         [   2,   13, 5256,  ...,    0,    0,    0],
         ...,
         [   2,  701,  148,  ...,    0,    0,    0],
         [   2, 1455,   70,  ...,    0,    0,    0],
         [   2,   42,  510,  ...,    0,    0,    0]], device='cuda:0'),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
         1, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')]

In [10]:
# 2.1 Import and load the AlbertModel class (albert-base-v2) from the HuggingFace library.
model = AlbertModel.from_pretrained('albert-base-v2')

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# The AlbertClassifier class

class AlbertClassifier(pl.LightningModule):
    def __init__(self, dropout_p, hid_dim, output_dim):
        super().__init__()
        self.albert = model
        self.dropout = torch.nn.Dropout(dropout_p)
        self.linear_1 = torch.nn.Linear(hid_dim, hid_dim)
        self.linear_2 = torch.nn.Linear(hid_dim, output_dim)
        self.loss = torch.nn.NLLLoss()

    def forward(self, input_ids, attention_mask, token_ids):
        x1 = self.albert(input_ids, attention_mask=attention_mask, token_type_ids=token_ids)[0]
        x1 = x1[:,0]
        x1 = self.dropout(torch.nn.ReLU()(self.linear_1(x1)))
        output  = torch.log_softmax(self.linear_2(x1), dim = 1)
        return output

    def training_step(self, batch, ix):
        pred = self(batch[0], batch[1], batch[2])
        loss = self.loss(pred, batch[3].view(-1))
        return loss

    def validation_step(self, batch, ix):
        pred = self(batch[0], batch[1], batch[2])
        loss = self.loss(pred, batch[3].view(-1))
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

m = AlbertClassifier(0.5, 768, 2)

device = "cuda"
t = pl.Trainer(max_epochs=3, gpus=1) # trainer() object in pytorch lightning
t.fit(m.to(device), dls)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type        | Params
-----------------------------------------
0 | albert   | AlbertModel | 11.7 M
1 | dropout  | Dropout     | 0     
2 | linear_1 | Linear      | 590 K 
3 | linear_2 | Linear      | 1.5 K 
4 | loss     | NLLLoss     | 0     
-----------------------------------------
12.3 M    Trainable params
0         Non-trainable params
12.3 M    Total params
49.103    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]